##### Import General Functions

In [49]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd

### Function to Run ODE - Deconstructing Fung et al., 2011

##### Import Functions and Dictionaries

In [ ]:
from ODESystem_RecreatingFung2011 import Fung2011ODE
from ODESystem_RecreatingFung2011 import Fung2011ODE_norec
from ODESystem_RecreatingFung2011 import Fung2011ODE_norec_noinh
from ODESystem_RecreatingFung2011 import Fung2011ODE_norec_noinh_nogp

from ODEDictionaries_RecreatingFung2011 import Fung_NUTR_d
from ODEDictionaries_RecreatingFung2011 import Fung_SED_d
from ODEDictionaries_RecreatingFung2011 import Fung_NUTR_norec_d
from ODEDictionaries_RecreatingFung2011 import Fung_SED_norec_d
from ODEDictionaries_RecreatingFung2011 import Fung_NUTR_noinh_d
from ODEDictionaries_RecreatingFung2011 import Fung_SED_noinh_d
from ODEDictionaries_RecreatingFung2011 import Fung_NUTR_nogp_d
from ODEDictionaries_RecreatingFung2011 import Fung_SED_nogp_d

In [10]:
def ODEfunction_DeconstructFung(
    NUMYEARS,
    time_points,  
    scenario,
    dictionary,
    function,
    xparam,
    init_cond):

    
    #init_cond
    B0, G0, S0 = init_cond['B0'], init_cond['G0'], init_cond['S0']
    

    outputs = []
    
    for x in xparam:
    
        for C0 in np.linspace(0.01,0.99,2):

            
            #Initial Conditions-----------------------------------------------------
            C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2) 
            B0, G0, S0 = B0, G0, S0

            
            #Fung et al., 2011 CTMm ODE System-----------------------------------------------------
            lCs = dictionary['lCs'] 
            lCb = dictionary['lCb'] 
            εC  = dictionary['εC'] 
            rC  = dictionary['rC'] 
            αCT = dictionary['αCT'] 
            μC  = dictionary['μC']
            rM  = dictionary['rM'] 
            βM  = dictionary['βM'] 
            αMT = dictionary['αMT'] 
            αMC = dictionary['αMC']
            rT  = dictionary['rT'] 
            gT  = dictionary['gT'] 
            gM  = dictionary['gM'] 
            θ   = dictionary['θ']

            P = {
                'lCs': lCs, 
                'lCb': lCb, 
                'εC': εC, 
                'rC' : rC, 
                'αCT': αCT, 
                'μC' : μC, 
                'rM' : rM, 
                'βM': βM, 
                'αMT': αMT, 
                'αMC': αMC, 
                'rT' : rT, 
                'gT': gT, 
                'gM': gM, 
                'θ': θ}

             
            #Solve ODE-----------------------------------------------------
            sol = solve_ivp(function, [0, NUMYEARS], [C0, M0, T0], 
                            method = 'RK45', args = (P,), dense_output=True)

            N = sol.sol(time_points)

            C_array = N[0,:]
            M_array = N[1,:]
            T_array = N[2,:]

            
            #Ensure Run Reached Equilibrium-----------------------------------------------------
            thr1 = 1e-06
            
            Cp = C_array[-1]
            Mp = M_array[-1]
            Tp = T_array[-1]
            
            Cp2 = C_array[-2]
            Mp2 = M_array[-2]
            Tp2 = T_array[-2]
           
            if abs(Cp - Cp2) < thr1 and abs(Mp - Mp2) < thr1 and abs(Tp - Tp2) < thr1:
                equil = 'yes'

            else:
                equil = 'no'


            #Categorize State at Equilibrium-----------------------------------------------------
            thr2 = 0.01
            Fp = 1 - (Cp + Mp + Tp)
            
            C_t = Cp >= thr
            M_t = Mp >= thr
            T_t = Fp >= thr

            if C_t and not M_t and not T_t:
                ben_col = "allC"
                
            elif M_t and not C_t and not T_t:
                ben_col = "allM"
            
            elif T_t and not C_t and not M_t:
                ben_col = "allT"
            
            elif not C_t and not M_t and not T_t:
                ben_col = "allF"
            
            else:
                dom_val = max(Cp, Mp, Tp, Fp)
            
                if dom_val == Cp:
                    ben_col = "mostlyC" if T_t else "mostlyC_noT"
            
                elif dom_val == Mp:
                    ben_col = "mostlyM" if T_t else "mostlyM_noT"
            
                elif dom_val == Tp:
                    ben_col = "mostlyT"
                
                elif dom_val == Fp:
                    ben_col = "mostlyF"
                    
                else:
                    ben_col = "other"

                        
            #Save Results-----------------------------------------------------
            benthos = (rC, αCT, μC, rM, αMT, αMC, rT, gT, gM)
            state_eq = (Cp, Mp, Tp)
            labels = (ben_col, equil)

            
            outputs.append(
                benthos
                + state_eq
                + labels
             )

            
            benthos_c = ('rC', 'αCT', 'μC', 'rM', 'αMT', 'αMC', 'rT', 'gT', 'gM')
            state_eq_c = ('Cp', 'Mp', 'Tp')
            labels_c = ('ben_col', 'equil')

            columns = (
                benthos_c
                + state_eq_c
                + labels_c
            )

        
    df_prime = pd.DataFrame(outputs, columns=columns)
          
    return(df_prime)

### Function to Run ODE - Building Snyder ODE from Basal Fung et al., 2011 Model

##### Import Functions and Dictionaries

In [ ]:
from ODESystem_RecreatingFung2011 import SnyderODE

#benthos
from ODEDictionaries_RecreatingFung2011 import NUTR_benthos
from ODEDictionaries_RecreatingFung2011 import SED_benthos

#browser
from ODEDictionaries_RecreatingFung2011 import B_NUTR_herb_d
from ODEDictionaries_RecreatingFung2011 import B_SED_herb_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_conv_d
from ODEDictionaries_RecreatingFung2011 import B_SED_conv_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_hand_d
from ODEDictionaries_RecreatingFung2011 import B_SED_hand_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_mort_d
from ODEDictionaries_RecreatingFung2011 import B_SED_mort_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_imm_d
from ODEDictionaries_RecreatingFung2011 import B_SED_imm_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_em_d
from ODEDictionaries_RecreatingFung2011 import B_SED_em_d
from ODEDictionaries_RecreatingFung2011 import B_NUTR_rec_d
from ODEDictionaries_RecreatingFung2011 import B_SED_rec_d

#grazer
from ODEDictionaries_RecreatingFung2011 import G_NUTR_herb_d
from ODEDictionaries_RecreatingFung2011 import G_SED_herb_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_conv_d
from ODEDictionaries_RecreatingFung2011 import G_SED_conv_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_hand_d
from ODEDictionaries_RecreatingFung2011 import G_SED_hand_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_mort_d
from ODEDictionaries_RecreatingFung2011 import G_SED_mort_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_imm_d
from ODEDictionaries_RecreatingFung2011 import G_SED_imm_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_em_d
from ODEDictionaries_RecreatingFung2011 import G_SED_em_d
from ODEDictionaries_RecreatingFung2011 import G_NUTR_rec_d
from ODEDictionaries_RecreatingFung2011 import G_SED_rec_d

#scraper
from ODEDictionaries_RecreatingFung2011 import S_NUTR_herb_d
from ODEDictionaries_RecreatingFung2011 import S_SED_herb_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_conv_d
from ODEDictionaries_RecreatingFung2011 import S_SED_conv_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_hand_d
from ODEDictionaries_RecreatingFung2011 import S_SED_hand_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_mort_d
from ODEDictionaries_RecreatingFung2011 import S_SED_mort_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_imm_d
from ODEDictionaries_RecreatingFung2011 import S_SED_imm_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_em_d
from ODEDictionaries_RecreatingFung2011 import S_SED_em_d
from ODEDictionaries_RecreatingFung2011 import S_NUTR_rec_d
from ODEDictionaries_RecreatingFung2011 import S_SED_rec_d

In [9]:
def ODEfunction_BuildSnyder(
    NUMYEARS,
    time_points,  
    init_cond,
    test_param,
    test_param_val,
    xparam,
    benthos_dictionary,
    herb_dictionary,
    function):

    #init_cond
    B0, G0, S0 = init_cond['B0'], init_cond['G0'], init_cond['S0']
    

    outputs = []

   
    for x in xparam:

        for test_param in test_param_val:
    
            for C0 in np.linspace(0.01,0.99,2):


                #Initial Conditions-----------------------------------------------------
                C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2) 
                B0, G0, S0 = B0, G0, S0

                #Benthos Parameters-----------------------------------------------------
                rC  = benthos_dictionary['rC']
                αCT = benthos_dictionary['αCT']
                μC  = benthos_dictionary['μC']
                rM  = benthos_dictionary['rM']
                αMT = benthos_dictionary['αMT']
                αMC = benthos_dictionary['αMC']
                rT  = benthos_dictionary['rT']

                #Herbivore Parameters-----------------------------------------------------
                s = herb_dictionary['s']
                h = herb_dictionary['h']
                η = herb_dictionary['η']
                e = herb_dictionary['e']
                μ = herb_dictionary['μ']
                θ = herb_dictionary['θ']
                γ = herb_dictionary['γ']
                φ = herb_dictionary['φ']

                
                test_param = test_param


                #Parameter Dictionary
                P = {
                    'rC' : rC,
                    'αCT': αCT,
                    'μC' : μC,
                    'rM' : rM,
                    'αMT': αMT,
                    'αMC': αMC,
                    'rT' : rT,
                    's'  : s,
                    'h'  : h,
                    'η'  : η,
                    'e'  : e,
                    'μ'  : μ,
                    'θ'  : θ,
                    'γ'  : γ,
                    'φ'  : φ
                }

                
                #Solve ODE-----------------------------------------------------
                sol = solve_ivp(function, [0,NUMYEARS], [C0, M0, T0, B0, G0, S0], 
                                method = 'RK45', args = (P,), dense_output=True)

                N = sol.sol(time_points)

                C_array = N[0,:]
                M_array = N[1,:]
                T_array = N[2,:]
                
                B_array = N[3,:]
                G_array = N[4,:]
                S_array = N[5,:]

                
                #Ensure Run Reached Equilibrium-----------------------------------------------------
                thr1 = 1e-06
            
                Cp = C_array[-1]
                Mp = M_array[-1]
                Tp = T_array[-1]

                Bp = B_array[-1]
                Gp = G_array[-1]
                Sp = S_array[-1]

                Cp2 = C_array[-2]
                Mp2 = M_array[-2]
                Tp2 = T_array[-2]

                Bp2 = B_array[-2]
                Gp2 = G_array[-2]
                Sp2 = S_array[-2]                
               
                if abs(Cp - Cp2) < thr1 and abs(Mp - Mp2) < thr1 and abs(Tp - Tp2) < thr1 and abs(Bp - Bp2) < thr1 and abs(Gp - Gp2) < thr1 and abs(Sp - Sp2) < thr1:
                    equil = 'yes'

                else:
                    equil = 'no'


                #Categorize State at Equilibrium-----------------------------------------------------
                thr2 = 0.01

                #Benthos
                Fp = 1 - (Cp + Mp + Tp)
                C_t = Cp >= thr
                M_t = Mp >= thr
                T_t = Fp >= thr

                if C_t and not M_t and not T_t:
                    ben_col = "allC"
                
                elif M_t and not C_t and not T_t:
                    ben_col = "allM"
                
                elif T_t and not C_t and not M_t:
                    ben_col = "allT"
                
                elif not C_t and not M_t and not T_t:
                    ben_col = "allF"
                
                else:
                    dom_val = max(Cp, Mp, Tp, Fp)
                
                    if dom_val == Cp:
                        ben_col = "mostlyC" if T_t else "mostlyC_noT"
                
                    elif dom_val == Mp:
                        ben_col = "mostlyM" if T_t else "mostlyM_noT"
                
                    elif dom_val == Tp:
                        ben_col = "mostlyT"
                    
                    elif dom_val == Fp:
                        ben_col = "mostlyF"
                        
                    else:
                        ben_col = "other"

                #Herbivores
                B_t = Bp >= thr
                G_t = Gp >= thr
                S_t = Sp >= thr

                if B_t and not G_t and not S_t:
                    herb_col = "allB"
                
                elif G_t and not B_t and not S_t:
                    herb_col = "allG"
                
                elif S_t and not B_t and not G_t:
                    herb_col = "allS"
                
                elif not B_t and not G_t and not S_t:
                    herb_col = "nofish"

                else:
                    dom_val = max(Bp, Gp, Sp)
                
                    if dom_val == Bp:
                        herb_col = "mostlyB"
                
                    elif dom_val == Gp:
                        herb_col = "mostlyG"
                
                    elif dom_val == Sp:
                        herb_col = "mostlyS"

                    else:
                        herb_col = "other"

        
                #Save Results-----------------------------------------------------
                benthos  = (rC, αCT, μC, rM, αMT, αMC, rT)
                herbivores = (s, h, η, e, μ, θ, γ, φ)
                state_eq = (Cp, Mp, Tp, Bp, Gp, Sp)
                labels   = (ben_col, herb_col, equil)
      
                                    
                outputs.append(
                    benthos
                    + herbivores
                    + state_eq
                    + labels
                 )
    
                
                benthos_c = ('rC', 'αCT', 'μC', 'rM', 'αMT', 'αMC', 'rT')
                herbivores_c = ('s', 'h', 'η', 'e', 'μ', 'θ', 'γ', 'φ')
                state_eq_c = ('Cp', 'Mp', 'Tp', 'Bp', 'Gp', 'Sp')
                labels_c = ('ben_col', 'herb_col', 'equil')

                columns = (
                    benthos_c
                    + herbivores_c
                    + state_eq_c
                    + labels_c
                )

        
    df_prime = pd.DataFrame(outputs, columns=columns)
          
    return(df_prime)